## Implement a classic martingale using Python and Pandas

It contains toss results (-1=lose 1=win)  

I would like to change stake (the amount I bet every bet) using classic martingale.  

Initial stake is 1.  

If I lose stake will be 2 times previous stake (multiplier=2).  

If I win stake will be stake_initial  

In [2]:
# pandas version    one slight change:mark a loss as a 1 and a win as a 0

import pandas as pd

import numpy as np

toss = np.random.randint(0,2,100)

toss = pd.Series(toss)

steps = (toss.cumsum() * toss).diff() # mask out the cumsum where we won [0 1 2 3 0 0 4 5 6 ...]
edges = steps < 0 # find where the cumsum steps down -> where we won
dsteps = steps[edges].diff() # find the length of each losing streak
dsteps[steps[edges].index[0]] = steps[edges][:1] # fix length of the first run which in now NaN
toss2 = toss.copy() # get a copy of the toss series
toss2[edges] = dsteps # insert the length of the losing streaks into the copy of the toss results
bets = 2 ** (toss2).cumsum() # compute the wagers

res = pd.DataFrame({'toss': toss,
                    'toss2': toss2,
                    'runs': toss2.cumsum(),
                    'next_bet': bets})

In [3]:
toss

0     1
1     1
2     1
3     0
4     0
     ..
95    0
96    1
97    0
98    0
99    0
Length: 100, dtype: int32

In [6]:
res

toss  toss2  runs  next_bet
0      1      1     1         2
1      1      1     2         4
2      1      1     3         8
3      0     -3     0         1
4      0      0     0         1
..   ...    ...   ...       ...
95     0     -1     0         1
96     1      1     1         2
97     0     -1     0         1
98     0      0     0         1
99     0      0     0         1

[100 rows x 4 columns]

In [5]:
# Numpy version
# This is the pure numpy version (my native language is it were)
# There is a bit of fineagling to get the arrays to line up that pandas does for you

toss = np.random.randint(0,2,100)

steps = np.diff(np.cumsum(toss) * toss)
edges = steps < 0
edges_shift = np.append(False, edges[:-1])
init_step = steps[edges][0]
toss2 = np.array(toss)
toss2[edges_shift] = np.append(init_step, np.diff(steps[edges]))
bets = 2 ** np.cumsum(toss2)

fmt_dict = {1:'l', 0:'w'}
for t, b in zip(toss, bets):
    print(fmt_dict[t] + '-> {0:d}'.format(b))

IndexError: boolean index did not match indexed array along dimension 0; dimension is 100 but corresponding boolean dimension is 99